In [ ]:
%pip install langchain openai transformers --quiet
%pip install -U langchain-openai --quiet
%pip install torch torchvision torchaudio --quiet
%pip install -U langchain-community --quiet
%pip install ipywidgets --quiet
%pip install pandas --quiet
%pip install scikit-learn --quiet

In [ ]:
import os
import pandas as pd
from PIL import Image as PILImage
from PIL import Image, ImageEnhance, ImageFilter
from transformers import CLIPProcessor, CLIPModel
import torch
from langchain.agents import initialize_agent
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage
from langchain.tools import BaseTool
from sentence_transformers import SentenceTransformer, util
import chromadb
import base64
from chromadb.utils.embedding_functions import OpenCLIPEmbeddingFunction
from chromadb.utils.data_loaders import ImageLoader
from IPython.display import Image, display, Markdown
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [ ]:
# Set the OpenAI API key
os.environ["OPENAI_API_KEY"] = "sk-proj-uRoYTQ11Qi4yndbUFWSOT3BlbkFJ5dcSHgZPnJRjSpXUwRU7"

# Initialise ChatGPT
gpt4o = ChatOpenAI(temperature=0, model_name="gpt-4o")
# gpt4o = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

# Initialise CLIP model and processor
clip_model_name = "openai/clip-vit-base-patch32"
clip_processor = CLIPProcessor.from_pretrained(clip_model_name)
clip_model = CLIPModel.from_pretrained(clip_model_name)

# Initialise the ChromaDB client
chroma_client = chromadb.PersistentClient(path="./image_vdb")

# Initialise ChromaDB Image Loader
image_loader = ImageLoader()
image_loader._PILImage = PILImage

# Initialize CLIP embedding
CLIP = OpenCLIPEmbeddingFunction()
# Create a database of image vectors
image_vdb = chroma_client.get_or_create_collection(name="image", embedding_function=CLIP, data_loader=image_loader)


In [ ]:
# Image Preprocessing Functions
def preprocess_image(image_path):
    image = PILImage.open(image_path).convert('RGB')

    # Resizing images
    image = image.resize((224, 224))

    # Enhanced contrast
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(2)

    # Denoising
    image = image.filter(ImageFilter.MedianFilter(size=3))

    return image

def extract_image_features(image_path, text):
    # Pre-processed images
    image_object = preprocess_image(image_path)

    # Image feature extraction using CLIP model
    clip_image_inputs = clip_processor(images=image_object, return_tensors="pt")
    clip_text_inputs = clip_processor(text=[text], return_tensors="pt")

    with torch.no_grad():
        clip_image_outputs = clip_model.get_image_features(**clip_image_inputs)
        clip_text_outputs = clip_model.get_text_features(**clip_text_inputs)

    # Normalize the features to have unit norm
    clip_image_outputs = torch.nn.functional.normalize(clip_image_outputs, p=2, dim=-1)
    clip_text_outputs = torch.nn.functional.normalize(clip_text_outputs, p=2, dim=-1)

    # Combine image and text features
    combined_features = (clip_image_outputs + clip_text_outputs) / 2

    # Spreading feature vectors into a one-dimensional list
    image_features = combined_features.flatten().tolist()

    return image_features


In [ ]:
def clear_vdb():
    # Delete the existing collection
    chroma_client.delete_collection(name="image")

    # Recreate the collection
    global image_vdb
    image_vdb = chroma_client.get_or_create_collection(
        name="image", 
        embedding_function=CLIP, 
        data_loader=image_loader
    )

def add_images_to_vdb(directory_paths, max_images_per_folder=2000):
    ids = []
    uris = []
    embeddings = [] 
    global_id = 0

    for directory_path in directory_paths:
        images_processed = 0
        for filename in sorted(os.listdir(directory_path)):
            if filename.lower().endswith('.jpg'):
                file_path = os.path.join(directory_path, filename)
                text = os.path.basename(os.path.dirname(file_path))
                print(f"Extracted text for {file_path}: {text}")
                image_features = extract_image_features(file_path, text)
                
                ids.append(str(global_id))
                uris.append(file_path)
                embeddings.append(image_features)
                global_id += 1
                images_processed += 1

                # Maximum number of images per folder max_images_per_folder
                if images_processed >= max_images_per_folder:
                    break

    # Add all images to the database at once
    if ids:
        image_vdb.add(ids=ids, uris=uris, embeddings=embeddings) 
        print(f"Added {len(ids)} images to the database.")

# Define the local image catalogue
local_image_dirs = [
    r'C:\Users\wanghanyu\Desktop\5_Classes\Fabricated',
    r'C:\Users\wanghanyu\Desktop\5_Classes\Forged',
    r'C:\Users\wanghanyu\Desktop\5_Classes\Other',
    r'C:\Users\wanghanyu\Desktop\5_Classes\Sheet Metal',
    r'C:\Users\wanghanyu\Desktop\5_Classes\Turned'
]

# Clear the vector database before adding new images
clear_vdb()

# Call function to add image to vector database
add_images_to_vdb(local_image_dirs)
print("Image count in VDB:", image_vdb.count())


In [ ]:
def query_db(image_path, text, results=3, top_k=500):
    query_features = extract_image_features(image_path, text)
    try:
        all_results = image_vdb.query(
            
            query_embeddings=[query_features],
            n_results=top_k + 1,
            include=['uris', 'distances']
        )
    except Exception as e:
        print(f"Error querying the database: {e}")
        return None
    
    filtered_results = filter_results(all_results, image_path, top_k=results)

    # 可视化结果
    # visualize_results(image_path, filtered_results['uris'][0])

    return filtered_results


def filter_results(all_results, image_path, top_k):
    # Combined query results
    distances_and_uris_and_ids = list(zip(all_results['distances'][0], all_results['uris'][0], all_results['ids'][0]))

    # Filter out results that query the image itself and have a distance of 0
    filtered_results = [
        (dist, uri, id) for dist, uri, id in distances_and_uris_and_ids
        if os.path.normpath(uri) != os.path.normpath(image_path) and dist != 0
    ]

    sorted_results = sorted(filtered_results, key=lambda x: x[0])

    unique_results = []
    seen_distances = set()
    for dist, uri, id in sorted_results:
        if dist not in seen_distances:
            unique_results.append((dist, uri, id))
            seen_distances.add(dist)


    final_results = unique_results[:top_k]

    sorted_distances = [dist for dist, _, _ in final_results]
    sorted_uris = [uri for _, uri, _ in final_results]
    sorted_ids = [id for _, _, id in final_results]

    return {
        'ids': [sorted_ids],
        'uris': [sorted_uris],
        'distances': [sorted_distances]
    }

def visualize_results(query_image_path, result_uris):
    fig, axs = plt.subplots(2, 2, figsize=(12, 12))
    fig.suptitle('Query Image and Top 3 Results', fontsize=16)

    query_img = PILImage.open(query_image_path).convert('RGB')
    axs[0, 0].imshow(query_img)
    axs[0, 0].set_title('Query Image')
    axs[0, 0].axis('off')

    for i, uri in enumerate(result_uris[:3]):
        img = PILImage.open(uri).convert('RGB')
        row = (i + 1) // 2
        col = (i + 1) % 2
        axs[row, col].imshow(img)
        axs[row, col].set_title(f'Result {i+1}')
        axs[row, col].axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
def format_prompt_inputs(data, user_query, text_info):
    inputs = {}
    inputs['user_query'] = user_query
    inputs['text_info'] = text_info

    # Acquire and encode the master image
    image_path_1 = data['uris'][0][0]
    with open(image_path_1, 'rb') as image_file:
        image_data_1 = image_file.read()
    inputs['image_data_1'] = base64.b64encode(image_data_1).decode('utf-8')

    # Extract and include retrieved text information
    retrieved_texts = [os.path.basename(os.path.dirname(uri)) for uri in data['uris'][0]]
    inputs['retrieved_info_texts'] = ', '.join(retrieved_texts)

    # Coding retrieved images
    for i, uri in enumerate(data['uris'][0][:3]): 
        with open(uri, 'rb') as image_file:
            inputs[f'retrieved_info_images_{i}'] = base64.b64encode(image_file.read()).decode('utf-8')

    # If fewer than 3 images are retrieved, fill with an empty string
    for i in range(len(data['uris'][0]), 3):
        inputs[f'retrieved_info_images_{i}'] = ''

    # Contains retrieved information
    inputs['retrieved_info'] = {
        'uris': data['uris'][0],
        'distances': data['distances'][0],
        'texts': retrieved_texts,
    }

    return inputs


# Initialise the output parser
parser = StrOutputParser()

image_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a mechanical engineer specializing in manufacturing processes. Your task is to analyze images of mechanical components, 
     identify their primary manufacturing method, and briefly describe key features of the component.

You will be provided with the main image to analyze, along with up to three similar components retrieved from a database. 
     Your goal is to use the additional images for comparison and to enhance your analysis, providing a more comprehensive and accurate description."""),

    ("user", [
        {"type": "text", "text": """Analyze the main image of the mechanical component and identify its primary manufacturing method and key features.

You have access to up to 3 similar components retrieved from the database, the retrieved images are displayed below.

Consider the similarities and differences between the main image and the retrieved images to provide a more accurate and comprehensive description.

Respond with a concise sentence stating the primary manufacturing method used to produce this component and key features."""},
        {"type": "image_url", "image_url": "data:image/jpeg;base64,{image_data_1}"},
        {"type": "image_url", "image_url": "data:image/jpeg;base64,{retrieved_info_images_0}"},
        {"type": "image_url", "image_url": "data:image/jpeg;base64,{retrieved_info_images_1}"},
        {"type": "image_url", "image_url": "data:image/jpeg;base64,{retrieved_info_images_2}"},
    ]),
])

# Define the LangChain chain
vision_chain = image_prompt | gpt4o | parser

def generate_description_with_rag(image_path):
    try:
        text_info = os.path.basename(os.path.dirname(image_path))
        user_query = "Analyze the mechanical component image with reference to similar components."
        results = query_db(image_path, text_info, results=3)
        print(f"Query results for {image_path}: {results}")
        
        if results is None or len(results['uris'][0]) == 0:
            print(f"Warning: No results retrieved for {image_path}")
            return "Unable to generate description due to lack of similar components."
        
        prompt_input = format_prompt_inputs(results, user_query, text_info)
        
        response = vision_chain.invoke(prompt_input)
        return response
    except Exception as e:
        print(f"Error in generate_description_with_rag for {image_path}: {str(e)}")
        return f"Error generating description: {str(e)}"

# Define a tool class for describing images
class DescribeImageWithDetailsTool(BaseTool):
    name = "Describe Image With Details Tool"
    description = 'Use this tool to describe an images of mechanical parts with additional details.'
    
    def _run(self, image_path: str):
        return self.run(image_path)
    
    def run(self, image_path: str):
        description = generate_description_with_rag(image_path)
        return description

# Initialise tools and agents
tools = [DescribeImageWithDetailsTool()]
agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=gpt4o,
    max_iterations=5,
    early_stopping_method='generate'
)

In [ ]:
def process_directories_for_testing(directory_paths, test_images_per_folder=50):
    results = []
    for directory_path in directory_paths:
        images_processed = 0
        for filename in sorted(os.listdir(directory_path))[50:50 + test_images_per_folder]:  
            if filename.lower().endswith('.jpg'):
                image_path = os.path.join(directory_path, filename)
                print(f"Processing test image: {image_path}")
                if os.path.exists(image_path):
                    try:
                        description = generate_description_with_rag(image_path)
                        result = {"image_name": filename, "description": description}
                        results.append(result)
                    except Exception as e:
                        results.append({"image_name": filename, "description": f"Error processing file: {str(e)}"})
                else:
                    results.append({"image_name": filename, "description": "File not found"})
        print(f"Processed image for testing from {directory_path}")

    output_file_path = os.path.join(os.getcwd(), "RAG_Future.csv")
    df = pd.DataFrame(results)
    df.to_csv(output_file_path, index=False)
    print(f"Test results saved at: {output_file_path}")

    return results

In [ ]:
# # Process the catalogue and generate test descriptions
test_results = process_directories_for_testing(local_image_dirs)

# Print the result
for result in test_results:
    print(f"{result['image_name']}: {result['description']}")